In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math

cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
classifier = Classifier(r"D:\OneDrive - corhuila.edu.co\TESIS 2\converted_keras (1)\keras_model.h5", r"D:\OneDrive - corhuila.edu.co\TESIS 2\converted_keras (1)\labels.txt")

offset = 20
imgSize = 300
labels = ["A", "B", "C"]  # Reemplazar con las 21 clases reales si están disponibles

while True:
    success, img = cap.read()
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

        aspectRatio = h / w
        if aspectRatio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            hCal = imgSize
            imgResize = cv2.resize(imgCrop, (wCal, hCal))

            if wCal > imgSize:
                wCal = imgSize
                imgResize = cv2.resize(imgCrop, (wCal, hCal))

            wGap = (imgSize - wCal) // 2
            imgWhite[:, wGap:wGap + wCal] = imgResize
        else:
            k = imgSize / w
            hCal = math.ceil(k * h)
            wCal = imgSize
            imgResize = cv2.resize(imgCrop, (wCal, hCal))

            if hCal > imgSize:
                hCal = imgSize
                imgResize = cv2.resize(imgCrop, (wCal, hCal))

            hGap = (imgSize - hCal) // 2
            imgWhite[hGap:hGap + hCal, :] = imgResize

        prediction, index = classifier.getPrediction(imgWhite, draw=False)

        cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                      (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
        cv2.putText(imgOutput, labels[index], (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
        cv2.rectangle(imgOutput, (x-offset, y-offset),
                      (x + w+offset, y + h+offset), (255, 0, 255), 4)

        cv2.imshow("ImageCrop", imgCrop)
        cv2.imshow("ImageWhite", imgWhite)

    cv2.imshow("Image", imgOutput)
    cv2.waitKey(1)

: 

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Parámetros
img_size = 224  # igual que Teachable Machine
batch_size = 32
dataset_dir = r"D:\OneDrive - corhuila.edu.co\TESIS\collect_images_letters_224x224"

# Preprocesamiento
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

train_gen = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    dataset_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Guardar etiquetas
class_names = list(train_gen.class_indices.keys())
with open("labels.txt", "w") as f:
    for label in class_names:
        f.write(label + "\n")

# Modelo estilo Teachable Machine
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento
history = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen
)

# Guardar modelo
model.save("modelo_21_clases.keras")


Found 1680 images belonging to 21 classes.
Found 420 images belonging to 21 classes.
Epoch 1/10
53/53 [==============================] - 46s 654ms/step - loss: 2.3106 - accuracy: 0.3327 - val_loss: 1.5673 - val_accuracy: 0.5762
Epoch 2/10
53/53 [==============================] - 34s 639ms/step - loss: 1.1132 - accuracy: 0.6905 - val_loss: 1.0521 - val_accuracy: 0.7262
Epoch 3/10
53/53 [==============================] - 34s 636ms/step - loss: 0.7688 - accuracy: 0.7810 - val_loss: 0.8823 - val_accuracy: 0.7643
Epoch 4/10
53/53 [==============================] - 32s 609ms/step - loss: 0.5540 - accuracy: 0.8458 - val_loss: 0.7696 - val_accuracy: 0.8333
Epoch 5/10
53/53 [==============================] - 33s 618ms/step - loss: 0.5070 - accuracy: 0.8583 - val_loss: 0.7287 - val_accuracy: 0.8024
Epoch 6/10
53/53 [==============================] - 33s 619ms/step - loss: 0.3677 - accuracy: 0.8946 - val_loss: 0.7106 - val_accuracy: 0.8143
Epoch 7/10
53/53 [==============================] - 33s 6